In [1]:
from plotly import graph_objects as go
import ipywidgets as widgets
from collections import deque
from copy import deepcopy

In [2]:
def add_circle(y): # dodadi krugcee prima kolona vo koja go stavame zetonot
    for i in range(M-1,-1,-1): # for ciklus koj odi od najvisok mozen indeks za redot do najnizok (toa e zosto zetonite pagjaat nadoli)
        if TABLE[i][y] == None: # dokolku ima bar edno prazno pole e prazno
            break
    return i, y # vrati pozicii na koe stavame zetonce
def game_end(board):
    if abs(board.x_check()) == 1000: # dokolku scoreto na tablata e 1000 i nekoj spoil po x
        return "Blue" if board.x_check() == 1000 else "Red" # proveri dali siniot spoil po x, ako ne crven
    if abs(board.y_check()) == 1000: # dali nekoj spoil po y -oska
        return "Blue" if board.y_check() == 1000 else "Red" # vidi dali e sin ili crven
    if abs(board.diag_check()) == 1000: # dali nekoj spoil po dijagonala
        return "Blue" if board.diag_check() == 1000 else "Red" # proveri dali e sin ili crven
    if abs(board.diag_check(False)) == 1000: # 2 dijagonala
        return "Blue" if board.diag_check(False) == 1000 else "Red"
    return None # ako nikoj ne spoil ne zavrshila igrata

In [3]:
def minimaxRoot(depth, board, player,node_count=0):
    # ova ne e alfa beta, gi razgranuva jazlite do kraj i se izvrshuva samo pri prv jazel- prv povik (pri sekoj povik taa tabla mu e pocetna)
    global moves # gi cuva site mozni potezi
    moves = board.available_moves() # go povikuva metodot avalible moves od Board
    scores = {} # prazen recnik za cuvanje na score vo sekoj poteg
    bestMove = -9999 if player else 9999 # Max igrac =1 , min igrac=0

    for move in moves: # iteracija za sekoj poteg vo listata na mozni potezi
        board.push(move) # pravi eden mozen poteg na originalnata tabla, vo for ciklusot 1 po 1 gi pravi site mozni potezi na tablata
        value,node_count,m,s= minimax(depth - 1, board, -10000, 10000, not player,node_count+1)# go povikuva minimaks ab so namalena dlabocina, eden jazel pomalku ja razgranuvanje ima i go menuva igracot
        print(f"Rekurzivno i vkupno, zosto ne pamtam shto izbrojav prviot pat koga me viknaa nikoj ne mi zacuva vo memorija:(, imam razgraneto :{node_count}")
        print(f"Jas sum od minimaksRoot")
        scores[move] = value # Go zacuvuva rezultatot vo recnikot scores
        board.pop()  # go vrakja potegot za da ja vrati tablata vo pocetna sostojba
        # OVIE DELOVI ZAKOMENTIRANI SE ALFA-BETA
        #if value == 1000:
         #   bestMove = value
          #  bestMoveFound = move
           # return bestMoveFound , [],[]
        if player and value >= bestMove: # Ako igracot e 1 = Max i ako imame poteg so podobar scorw od -99999
            bestMove = value # bestMove se postavuva na value
            bestMoveFound = move # najdobriot poteg e onoj za koj ispituvame
        elif not player and value <= bestMove: # istoto samo za min
            bestMove = value
            bestMoveFound = move
    
    return bestMoveFound ,node_count, moves,scores # vrati najdobar poteg,site mozni potezi, recnik za score za sekoj poteg

def minimax(depth, board, alpha, beta, player):
    if depth == 0: # dokolku nema povekje jazli za istazuvanje 
        return board.score() # vrati ja momentalnata sostojba na tabelata

    moves = board.available_moves() # se povikuva avalible moves od board
   # if not moves:
    #    return 1000

    if (player): # ako igracot e MAX
        bestMove = -9999
        for move in moves: # za sekoj poteg vo mmoznite potezi
            board.push(move) # napravi go potegot koj e na red
            move,node_count=minimax(depth - 1, board, alpha, beta, not player,node_count+1)
            bestMove=max(bestMove,move)
            print(f"Jas razgraniv rekurzivno : {node_count} jazli ")
            print(f"Jas sum od minimaks so alfa beta potsktatuvanje ama mene root me vika")
           # bestMove = max(bestMove, minimax(depth - 1, board, alpha, beta, not player)) # rekurzivno povikaj se i vidi dali best move e pogolem od momentalniot
            board.pop() # vrati ja tablata kako shto bese
            alpha = max(alpha, bestMove) # alfa postavi ja na pogolemata vrednost od alpha i best move
            if beta <= alpha: # ako beta e pomalo/ednakvo na alfa
                return bestMove # vrati najdobar poteg koj min kje go izbere, delot so alfa beta potskratuvanje e ovoj
        return bestMove # ako nema takov samo vrati go najdobriot poteg
    else: # Dkolku igracot ne e max mora da e min
        bestMove = 9999
        for move in moves: # za poteg vo moznite potezi
            board.push(move); # napravi go potegot
            bestMove = min(bestMove, minimax(depth - 1, board, alpha, beta, not player)) # rekurzivno povikaj se i vidi dali taa table te vodi do pomal score
            board.pop() # vrati se na pocetnata tabla
            beta = min(beta, bestMove) # izberi go beta da e najmalo mozno
            if beta <= alpha: # dali beta e pomalo ili ednakvo na alfa
                return bestMove # vrati go toj poteg - potkastruvanje
        return bestMove,node_count # dokolku ne toa e polosho z amin i vrati go najdobriot poteg

In [4]:
from copy import deepcopy  # Вклучува копирање на објекти, за да можеме да направиме независна копија на таблата.

class Board:  # Дефинира класа Board која ја претставува таблата на играта.
    def __init__(self, table, turn):  # Конструктор кој иницијализира табла и моментален ред на играчот (True за еден играч, False за другиот).
        self.table = deepcopy(table)  # Копира табла за да избегнеме модификација на оригиналната табла.
        self.queue = []  # Користи ред за следење на движењата.
        self.turn = turn  # Иницијализира кој играч е на ред (True или False).

    def push(self, move):  # Функција за додавање на ново движење во редот.
        self.queue.append(move)  # Додава движење во редот.
        self.update_table()  # Обновува табла по движењето.
        self.turn = not self.turn  # Превртува на редот на другиот играч.

    def pop(self):  # Функција за отстранување на последното движење.
        move = self.queue.pop()  # Отстранува последното движење од редот.
        self.update_table(move)  # Обновува табла по отстранувањето на движењето.
        self.turn = not self.turn  # Превртува на редот на другиот играч.
        return move  # Враќа го отстранетото движење.

    def peek(self):  # Функција за поглед на последното движење без да го отстраниме.
        return self.queue[-1]  # Враќа го последното движење во редот.

    def update_table(self, move=None):  # Функција за обновување на таблата по секое движење.
        if move != None:  # Ако има движење за отстранување.
            for x in range(M-1, -1, -1):  # Пребарување низ редовите од долу нагоре.
                if self.table[x][move] == None:  # Ако клетката е празна.
                    break  # Прекинуваме пребарувањето.
            if self.table[x][move] != None:  # Ако клетката е веќе пополнета.
                self.table[x][move] = None  # Поставуваме ја на None (празна).
            else:
                self.table[x + 1][move] = None  # Поставуваме ја на празно местото.
        else:  # Ако нема дадено движење, тоа значи дека треба да се додаде ново движење.
            y = self.peek()  # Поглед на последното движење во редот.
            for x in range(M - 1, -1, -1):  # Пребарување низ редовите од долу нагоре.
                if self.table[x][y] == None:  # Ако клетката е празна.
                    break  # Прекинуваме пребарувањето.
            self.table[x][y] = self.turn  # Поставуваме ново движење на таблата, зависно од тоа кој играч е на ред.

    def available_moves(self):  # Функција која ги наоѓа сите достапни движења.
        moves = []  # Празен список за достапни движења.
        for y in range(N):  # Пребарување низ колоните.
            if self.table[0][y] == None:  # Ако првиот ред на колоната е празен.
                moves.append(y)  # Додава ја колоната како доступно движење.
        return moves  # Враќа ја листата на достапни движења.

    def score(self):  # Функција која пресметува резултат.
        score = 0  # Почетен резултат.
        score += self.x_check()  # Проверка по X оска.
        score += self.y_check()  # Проверка по Y оска.
        score += self.diag_check()  # Проверка на главната дијагонала.
        score += self.diag_check(False)  # Проверка на споредната дијагонала.
        return score  # Враќа го вкупниот резултат.

    # Проверка по X оска (редови)
    def x_check(self):
        score = 0  # Почетен резултат за проверката по X.
        prices = [10, 20, 40, 1000]  # Награди за различни должини на серија.
        blue, red, blue_start, red_start = -1, -1, -1, -1  # Променливи за следење на игри во серија.
        for x in range(M):  # Пребарување по редови.
            left = 1 + len([y for y in range(N) if self.table[x][y] == None])  # Број на празни места во редот.
            for y in range(N):  # Пребарување по колоните.
                if red == 3:  # Ако има серија од 4 црвени (порака за загуба).
                    return -1000
                if blue == 3:  # Ако има серија од 4 сини (порака за победа).
                    return 1000
                # Проверка за премин од син (True) на црвен (False)
                if self.table[x][y] == True:  # Прекин на сината серија.
                    blue += 1
                    if blue == 0:
                        blue_start = y
                    if red >= 0:
                        score -= 0 if self.table[x][red_start-1] == True or red_start == 0 else prices[red]/(2*left)
                        red, red_start = -1, -1
                elif self.table[x][y] == False:  # Прекин на црвената серија.
                    red += 1
                    if red == 0:
                        red_start = y
                    if blue >= 0:
                        score += 0 if self.table[x][blue_start-1] == False or blue_start == 0 else prices[blue]/(2*left)
                        blue, blue_start = -1, -1
                elif self.table[x][y] == None:  # Ако има празно место, прераспределува награди.
                    if blue >= 0:
                        score += prices[blue]/(2*left) if self.table[x][blue_start-1] == False or blue_start == 0 else prices[blue]/left        
                    if red >= 0:
                        score -= prices[red]/(2*left) if self.table[x][red_start-1] == True or red_start == 0 else prices[red]/left
                    blue, red, blue_start, red_start = -1, -1, -1, -1

        return score

    # Проверка по Y оска (колони)
    def y_check(self):
        score = 0  # Почетен резултат за проверката по Y.
        prices = [4, 9, 19, 1000]  # Награди за различни должини на серија.
        blue, red, blue_start, red_start = -1, -1, -1, -1  # Променливи за следење на игри во серија.
        for y in range(N):  # Пребарување по колоните.
            left = 1  # Останати празни места во колоната.
            for x in range(M-1, -1, -1):  # Пребарување низ редовите од долу нагоре.
                if red == 3:  # Ако има серија од 4 црвени (порака за загуба).
                    return -1000
                if blue == 3:  # Ако има серија од 4 сини (порака за победа).
                    return 1000
                if self.table[x][y] == True:  # Прекин на сината серија.
                    blue += 1
                    if blue == 0:
                        blue_start = x
                    if red >= 0:
                        red, red_start = -1, -1
                elif self.table[x][y] == False:  # Прекин на црвената серија.
                    red += 1
                    if red == 0:
                        red_start = x
                    if blue >= 0:
                        blue, blue_start = -1, -1
                elif self.table[x][y] == None:  # Ако има празно место, прераспределува награди.
                    if blue >= 0:
                        score += prices[blue]/left
                    if red >= 0:
                        score -= prices[red]/left
                    blue, red, blue_start, red_start = -1, -1, -1, -1

        return score

    # Проверка на дијагонала (главна или споредна)
    def diag_check(self, main=True):
        score = 0  # Почетен резултат за проверка на дијагонала.
        blue, red = -1, -1  # Следење на сериите на сини и црвени.
        row, col, x, y = 0, 0, 0, 0  # Индекси за движење низ таблата.
        table = self.table[::-1] if main else self.table  # Ако е главна дијагонала, обрнува таблата.

        while row + 1 != M or col != N:  # Пребарување низ сите клетки на таблата.
            if table[x][y] == True:  # Прекин на сината серија.
                blue += 1
                if red >= 0: 
                    red = -1  # Прекин на црвената серија.
            elif table[x][y] == False:  # Прекин на црвената серија.
                red += 1
                if blue >= 0:
                    blue = -1  # Прекин на сината серија.
            elif table[x][y] == None:  # Празно место, реиницијализира сериите.
                blue, red = -1, -1

            if red == 3:  # Ако има серија од 4 црвени (порака за загуба).
                return -1000
            if blue == 3:  # Ако има серија од 4 сини (порака за победа).
                return 1000

            # Превод низ клетките.
            x -= 1
            y += 1
            if x < 0 and row + 1 == M:  # Преминување на следната колона.
                col += 1
                x = row
                y = col
                blue, red = -1, -1
            elif x < 0 and row + 1 < M:  # Преминување на редот.
                row += 1
                x = row
                y = col
                blue, red = -1, -1
            elif y > N - 1:  # Преминување на колоната.
                col += 1
                y = col
                x = row
                blue, red = -1, -1

        return score


In [5]:
turn = True
debug = True
def update_fig(trace, points, selector):
    x, y = add_circle(points.xs[0])
    global turn
    TABLE[x][y] = turn
    turn = not turn
    
    board = Board(TABLE,False)
    y, moves,scores = minimaxRoot(4, board, False)
    if debug : 
        print([(move,scores[move]) for move in moves])
    x,y = add_circle(y)
    TABLE[x][y] = turn
    turn = not turn
    colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.data[0].marker['color'] = colors
    
    if game_end(Board(TABLE,True)) != None:
        text.value = f'<font size=4> {game_end(Board(TABLE,True))} wins !!'
        fig.data[0].on_click(None)


def create_fig():
    fig = go.FigureWidget()
    x = [x for y in range(M) for x in range(N)]
    y = [y for y in range(M) for x in range(N)]
    colors = ['#121212' if n == None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.add_scatter(x=x, y=y, mode='markers', marker_size=48, 
        marker_symbol='circle', marker_color=colors,hoverinfo='none')
    fig.data[0].on_click(update_fig)
    fig.update_xaxes(range=[-0.5, N - 0.5], dtick=1, side='top', visible=False,autorange=False)
    fig.update_yaxes(range=[-0.5, M - 0.5], dtick=1, autorange='reversed', visible=False)
    fig.update_layout(width=80*N, height=80*N, showlegend=False,margin={'r':0,'l':0,'t':0,'b':0},plot_bgcolor='#212121')

    return fig

N = 7
M = 6
TABLE = [
    [None, None, None, None, None, None, None],
    [None, None, None, None, None, None, None],
    [None, None, None, None, None, None, None],
    [None, None, None, None, None, None, None],
    [None, None, None, None, None, None, None],
    [None, None, None, None, None, None, None],
]
fig = create_fig()
text = widgets.HTML(value='<font size=4> Connect 4')
widgets.VBox([fig,text])

    'data': [{'hoverinfo': 'none',
              'marker': {'color': [#121212, #…